In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from ppi_py import ppi_ols_ci
from ppi_py import ppi_ols_pointestimate
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
df=pd.read_csv('cyanobacteria_thermotolerance_amino_acid_composition_wide.csv')

In [4]:
amino_acids = ['A', 'R', 'N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
# Normalize by total AA count
df['total_aa'] = df[amino_acids].sum(axis=1)
for aa in amino_acids:
    df[f'{aa}_freq'] = df[aa] / df['total_aa']

high_quality_df = df[df['ogt'].notnull()].copy()
low_quality_df = df[df['genome_type']=='single_cell'].copy()

In [6]:
# Drop one amino acid (e.g., 'A')
aa_freq_cols = [f'{aa}_freq' for aa in amino_acids if aa != 'A']

X = high_quality_df[aa_freq_cols].values
y = high_quality_df['ogt'].values


## linear regression

In [9]:
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred.append(model.predict(X_test)[0])
    y_true.append(y_test[0])

# Evaluate
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}")


MAE: 2.043, RMSE: 2.782


In [27]:

from sklearn.model_selection import cross_val_score


# Assume X is a DataFrame of amino acid frequencies (each row sums to 1)
# and y is a continuous outcome

def clr_transform(X):
    # X: DataFrame or 2D numpy array with strictly positive values
    X = np.asarray(X)
    if np.any(X <= 0):
        raise ValueError("CLR input must be strictly positive")
    log_X = np.log(X)
    gm = np.mean(log_X, axis=1, keepdims=True)
    return log_X - gm


X_clr = clr_transform(X)  # Apply CLR transform


from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for train_idx, test_idx in loo.split(X_clr):
    model.fit(X_clr[train_idx], y[train_idx])
    pred = model.predict(X_clr[test_idx])[0]
    y_true.append(y[test_idx][0])
    y_pred.append(pred)
    
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}")


MAE: 1.486, RMSE: 1.949


## linear regression with PPI

In [11]:
n = len(y)
y_true_all, y_pred_ppi_all, y_pred_naive_all = [], [], []

for i in range(n):
    # 1. Leave one out
    X_test = X[i].reshape(1, -1)
    y_test = y[i]
    
    X_train = np.delete(X, i, axis=0)
    y_train = np.delete(y, i, axis=0)

    # 2. Simulate PPI: randomly pick 10 as labeled
    np.random.seed(i)  # for reproducibility
    idx = np.random.permutation(len(X_train))
    labeled_idx = idx[:10]
    unlabeled_idx = idx[10:]

    X_labeled = X_train[labeled_idx]
    y_labeled = y_train[labeled_idx]
    X_unlabeled = X_train[unlabeled_idx]
    yhat_labeled = LinearRegression().fit(X_labeled, y_labeled).predict(X_labeled)
    yhat_unlabeled = LinearRegression().fit(X_labeled, y_labeled).predict(X_unlabeled)

    # 3. Apply PPI
    ppi_beta = ppi_ols_pointestimate(X_labeled, y_labeled, yhat_labeled, X_unlabeled, yhat_unlabeled)
    intercept = LinearRegression().fit(X_labeled, y_labeled).intercept_

    # 4. Predict with PPI
    y_pred_ppi = X_test @ ppi_beta + intercept

    # 5. Predict with Naive
    naive_model = LinearRegression().fit(X_labeled, y_labeled)
    y_pred_naive = naive_model.predict(X_test)

    # 6. Collect predictions
    y_true_all.append(y_test)
    y_pred_ppi_all.append(y_pred_ppi.item())
    y_pred_naive_all.append(y_pred_naive.item())

# 7. Report metrics
mae_ppi = mean_absolute_error(y_true_all, y_pred_ppi_all)
mae_naive = mean_absolute_error(y_true_all, y_pred_naive_all)
rmse_ppi = mean_squared_error(y_true_all, y_pred_ppi_all, squared=False)
rmse_naive = mean_squared_error(y_true_all, y_pred_naive_all, squared=False)

print(f"PPI:    MAE = {mae_ppi:.2f}, RMSE = {rmse_ppi:.2f}")
print(f"Naive:  MAE = {mae_naive:.2f}, RMSE = {rmse_naive:.2f}")

PPI:    MAE = 1209.84, RMSE = 1832.79
Naive:  MAE = 3.01, RMSE = 4.33


In [31]:
n = len(y)
y_true_all, y_pred_ppi_all, y_pred_naive_all = [], [], []

for i in range(n):
    # 1. Leave one out
    X_test = X_clr[i].reshape(1, -1)
    y_test = y[i]
    
    X_train = np.delete(X_clr, i, axis=0)
    y_train = np.delete(y, i, axis=0)

    # 2. Simulate PPI: randomly pick 10 as labeled
    np.random.seed(i)  # for reproducibility
    idx = np.random.permutation(len(X_train))
    labeled_idx = idx[:10]
    unlabeled_idx = idx[10:]

    X_labeled = X_train[labeled_idx]
    y_labeled = y_train[labeled_idx]
    X_unlabeled = X_train[unlabeled_idx]
    yhat_labeled = LinearRegression().fit(X_labeled, y_labeled).predict(X_labeled)
    yhat_unlabeled = LinearRegression().fit(X_labeled, y_labeled).predict(X_unlabeled)

    # 3. Apply PPI
    ppi_beta = ppi_ols_pointestimate(X_labeled, y_labeled, yhat_labeled, X_unlabeled, yhat_unlabeled)
    intercept = LinearRegression().fit(X_labeled, y_labeled).intercept_

    # 4. Predict with PPI
    y_pred_ppi = X_test @ ppi_beta + intercept

    # 5. Predict with Naive
    naive_model = LinearRegression().fit(X_labeled, y_labeled)
    y_pred_naive = naive_model.predict(X_test)

    # 6. Collect predictions
    y_true_all.append(y_test)
    y_pred_ppi_all.append(y_pred_ppi.item())
    y_pred_naive_all.append(y_pred_naive.item())

# 7. Report metrics
mae_ppi = mean_absolute_error(y_true_all, y_pred_ppi_all)
mae_naive = mean_absolute_error(y_true_all, y_pred_naive_all)
rmse_ppi = mean_squared_error(y_true_all, y_pred_ppi_all, squared=False)
rmse_naive = mean_squared_error(y_true_all, y_pred_naive_all, squared=False)

print(f"PPI:    MAE = {mae_ppi:.2f}, RMSE = {rmse_ppi:.2f}")
print(f"Naive:  MAE = {mae_naive:.2f}, RMSE = {rmse_naive:.2f}")

PPI:    MAE = 558.26, RMSE = 814.82
Naive:  MAE = 3.79, RMSE = 7.03


## random forest

In [15]:
loo = LeaveOneOut()
y_true_all = []
y_pred_all = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)[0]

    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred)

# Evaluate
mae = mean_absolute_error(y_true_all, y_pred_all)
rmse = mean_squared_error(y_true_all, y_pred_all, squared=False)

print(f"Random Forest LOOCV → MAE: {mae:.2f}, RMSE: {rmse:.2f}")

Random Forest LOOCV → MAE: 1.21, RMSE: 1.69


In [28]:
loo = LeaveOneOut()
y_true_all = []
y_pred_all = []
X_clr = clr_transform(X)
for train_index, test_index in loo.split(X):
    X_train, X_test = X_clr[train_index], X_clr[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)[0]

    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred)

# Evaluate
mae = mean_absolute_error(y_true_all, y_pred_all)
rmse = mean_squared_error(y_true_all, y_pred_all, squared=False)

print(f"Random Forest LOOCV → MAE: {mae:.2f}, RMSE: {rmse:.2f}")

Random Forest LOOCV → MAE: 1.27, RMSE: 1.73


## boosting

In [25]:
from xgboost import XGBRegressor


loo = LeaveOneOut()
y_true_all = []
y_pred_all = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42,
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)[0]
    
    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred)

# Evaluate
mae = mean_absolute_error(y_true_all, y_pred_all)
rmse = mean_squared_error(y_true_all, y_pred_all, squared=False)

print(f"XGBoost LOOCV - MAE: {mae:.2f}, RMSE: {rmse:.2f}")


XGBoost LOOCV - MAE: 1.32, RMSE: 1.85


In [30]:
from xgboost import XGBRegressor


loo = LeaveOneOut()
y_true_all = []
y_pred_all = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X_clr[train_index], X_clr[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42,
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)[0]
    
    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred)

# Evaluate
mae = mean_absolute_error(y_true_all, y_pred_all)
rmse = mean_squared_error(y_true_all, y_pred_all, squared=False)

print(f"XGBoost LOOCV - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

XGBoost LOOCV - MAE: 1.49, RMSE: 1.95
